### 구글 드라이브 마운트

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
colab_default_dir = '/content/gdrive/My Drive/Colab Notebooks/'
working_dir = 'data'
os.chdir(colab_default_dir)
os.chdir(working_dir)

### 필요한 라이브러리 불러오기

In [ ]:
!pip install -q finance-datareader
!pip install -q mplfinance

In [3]:
import warnings; warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from collections import Counter
import yfinance as yf
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
from datetime import datetime
import sys
from mplfinance.original_flavor import candlestick_ohlc
import pandas_datareader.data as web
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from sklearn.model_selection import train_test_split

### 데이터 불러오기

In [4]:
df_01 = pd.read_csv("./NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202301.csv", encoding='cp949')
df_02 = pd.read_csv("./NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202302.csv", encoding='cp949')
df_03 = pd.read_csv("./NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202303.csv", encoding='cp949')
df_04 = pd.read_csv("./NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202304.csv", encoding='cp949')
df_05 = pd.read_csv("./NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202305.csv", encoding='cp949')
df_06 = pd.read_csv("./NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202306.csv", encoding='cp949')
df_07 = pd.read_csv("./NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202307.csv", encoding='cp949')
df_08 = pd.read_csv("./NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202308.csv", encoding='cp949')
df_tck = pd.read_csv("./NASDAQ_FC_STK_IEM_IFO.csv", encoding='cp949')
target = pd.read_csv("./NASDAQ_DT_FC_STK_QUT.csv", encoding='cp949')

### 뉴스 전처리

In [5]:
# 1~8월 뉴스 병합

df_01.drop_duplicates(inplace=True)
df_01 = df_01.reset_index(drop=True)

df_02.drop_duplicates(inplace=True)
df_02 = df_02.reset_index(drop=True)

df_03.drop_duplicates(inplace=True)
df_03 = df_03.reset_index(drop=True)

df_04.drop_duplicates(inplace=True)
df_04 = df_04.reset_index(drop=True)

df_05.drop_duplicates(inplace=True)
df_05 = df_05.reset_index(drop=True)

df_06.drop_duplicates(inplace=True)
df_06 = df_06.reset_index(drop=True)

df_07.drop_duplicates(inplace=True)
df_07 = df_07.reset_index(drop=True)

df_08.drop_duplicates(inplace=True)
df_08 = df_08.reset_index(drop=True)

df_news = pd.concat([df_01, df_02, df_03, df_04, df_05, df_06, df_07, df_08])
df_news = df_news.reset_index(drop=True)

In [6]:
# 모든 글자가 대문자인 제목이 있다. 이 경우엔 통째로 소문자로 바꾼다.

for i in range(len(df_news)):
    if df_news['til_ifo'][i].isupper():
        df_news.iloc[i,2] = df_news['til_ifo'][i].lower()

In [7]:
# 뉴스 제목과 요약 정보를 합친 열 생성

news = []
for i in range(len(df_news)):
    news.append(df_news['til_ifo'][i] + "." + " " + df_news['news_smy_ifo'][i])
df_news['combined_news'] = news

In [ ]:
# HTML 태그 제거

def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df_news['combined_news'] = df_news['combined_news'].apply(remove_html_tags)

In [ ]:
# 기업명을 티커 코드로 변경

df_tck['fc_sec_eng_nm'] = df_tck['fc_sec_eng_nm'].str.rstrip()
df_tck['tck_iem_cd'] = df_tck['tck_iem_cd'].str.rstrip()

## "기업명 (티커 코드)" -> "티커 코드"
corp_first = []
for i in range(len(df_tck)):
    corp_first.append(df_tck['fc_sec_eng_nm'][i].split(" ")[0])

for i, row in df_tck.iterrows():
    tck = row['tck_iem_cd']
    name_first = corp_first[i]
    df_news['combined_news'] = df_news['combined_news'].apply(lambda text: re.sub(f'{re.escape(name_first)}.*?{re.escape(f"({tck})")}', tck, text) if f"({tck})" in text else text)

## "기업명" -> "티커 코드"
for i, row in df_tck.iterrows():
    corp_name = row['fc_sec_eng_nm']
    tck_name = row['tck_iem_cd']
    df_news['combined_news'] = df_news['combined_news'].str.replace(corp_name, tck_name)

In [ ]:
# 괄호로 둘러싸인 부분 모두 제거

parentheses = []
pattern = r'\(.*?\)'

for i in range(len(df_news)):
    text = df_news['combined_news'][i]
    matches = re.findall(pattern, text)
    parentheses.extend(matches)

for pare in parentheses:
    df_news['combined_news'] = df_news['combined_news'].apply(lambda text: text.replace(pare, ""))

In [ ]:
# "."이 있는 줄임말을 수정한다.

for i in range(len(df_news)):
    if "Inc." in df_news['combined_news'][i]:
        df_news.iloc[i,8] = df_news['combined_news'][i].replace("Inc.", "Company")
    elif "INC." in df_news['combined_news'][i]:
        df_news.iloc[i,8] = df_news['combined_news'][i].replace("INC.", "Company")
    elif "Co." in df_news['combined_news'][i]:
        df_news.iloc[i,8] = df_news['combined_news'][i].replace("Co.", "Company")
    elif "CO." in df_news['combined_news'][i]:
        df_news.iloc[i,8] = df_news['combined_news'][i].replace("CO.", "Company")
    elif "Corp." in df_news['combined_news'][i]:
        df_news.iloc[i,8] = df_news['combined_news'][i].replace("Corp.", "Company")
    elif "CORP." in df_news['combined_news'][i]:
        df_news.iloc[i,8] = df_news['combined_news'][i].replace("CORP.", "Company")

In [ ]:
# 불용어 제거

## nltk가 정의한 불용어(일부 제외)
stop_words_list = stopwords.words('english')
stop_words_list.remove('above')
stop_words_list.remove('below')
stop_words_list.remove('up')
stop_words_list.remove('down')
stop_words_list.remove('over')
stop_words_list.remove('under')

## 길이가 2 이하이면서 빈도가 2 이하인 단어
tokenizer = TreebankWordTokenizer()
words = []
for i in range(len(df_news)):
    text = df_news['combined_news'][i]
    tokenized = tokenizer.tokenize(text)
    words.extend(tokenized)

element_count = Counter(words)
for element, count in element_count.items():
    if count <= 2:
        if len(element) <= 2: stop_words_list.append(element)
stop_words_list = set(stop_words_list)

## 제거
df_news['combined_news'] = df_news['combined_news'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_list]))

In [ ]:
# 숫자와 특수문자 제거

def remove_numbers_and_special_characters(text):
    cleaned_text = re.sub(r'[^A-Za-z\s]', ' ', text)
    return cleaned_text

df_news['combined_news'] = df_news['combined_news'].apply(remove_numbers_and_special_characters)

In [ ]:
# 공백 기준 길이가 1인 단어 제거

one_word = [word for i in range(len(df_news))
             for word in df_news['combined_news'][i].split() if len(word) == 1]

for i in range(len(df_news)):
    text = df_news['combined_news'][i]
    for word in one_word:
        text = re.sub(rf'\b{re.escape(word)}\b', '', text)
    df_news.iloc[i,8] = text

In [ ]:
# 연속된 공백을 1개의 공백으로 치환

def replace_multiple_spaces(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text

df_news['combined_news'] = df_news['combined_news'].apply(replace_multiple_spaces)

### 기술지표를 이용하여 긍정/부정/중립 분류

In [8]:
# NASDAQ_DT_FC_STK_QUT

## 열 이름 재설정
target.columns = ['거래일자', '티커종목코드', '종목시가', '종목고가', '종목저가', '종목종가', '누적거래수량', '매도체결합계수량', '매수체결합계수량']

## 거래일자를 날짜 형식으로 변환
target['거래일자'] = pd.to_datetime(target['거래일자'], format='%Y%m%d')

## 거래일자를 인덱스로 설정
target.set_index('거래일자', inplace=True)

In [ ]:
# 22년 12월 NASDAQ 데이터 불러오기

df_krx = fdr.StockListing('NASDAQ')

list1 = target['티커종목코드'].unique()
df_12_result = pd.DataFrame()

for symbol in list1:
    try:
        df_12 = yf.download(symbol, start='2022-12-01', end='2022-12-31', progress=False)
        if not df_12.empty:
            df_12['Symbol'] = symbol
            df_12_result = pd.concat([df_12_result, df_12])
    except: pass

df_12_result.columns = ['종목시가','종목고가','종목저가','종목종가','Adj Close','거래량','티커종목코드']
target = pd.concat([df_12_result,target])
target = target.drop(['Adj Close'], axis = 'columns')

In [ ]:
# 뉴스 데이터에는 있지만 NASDAQ_DT_FC_STK_QUT 데이터에는 없는 종목 추가

news_ticker_list = df_news['tck_iem_cd'].unique()
price_ticker_list =target['티커종목코드'].unique()
sub_set2 = [x for x in news_ticker_list if x not in price_ticker_list]
df_524_result = pd.DataFrame()

for symbol in sub_set2:
    try:
        df_524 = yf.download(symbol, start='2022-12-01', end='2023-08-31', progress=False)
        if not df_524.empty:
            df_524['티커코드'] = symbol
            df_524_result = pd.concat([df_524_result, df_524])
    except: pass

df_524_result.columns = ['종목시가','종목고가','종목저가','종목종가','Adj Close','거래량','티커종목코드']
target = pd.concat([df_524_result, target])
target = target.drop(['Adj Close'], axis = 'columns')

In [11]:
# 기술지표 구하기

code = list(target['티커종목코드'].unique())

## RSI
def calculate_RSI(data, code, period=14, column='종목종가'):
    results = []

    for i in code:
        subset = data[data['티커종목코드'] == i]
        delta = subset[column].diff(1)
        delta = delta.dropna()
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        avg_gain = up.rolling(window=period).mean()
        avg_loss = abs(down).rolling(window=period).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        subset['RSI'] = rsi
        results.append(subset)

    return pd.concat(results)

df_RSI = calculate_RSI(target,code, period = 14, column='종목종가')

## CMO
def CMO(df,code, n_days=20):
    results = []

    for i in code:
        subset = df[df['티커종목코드'] == i]
        delta = subset['종목종가'].diff()
        dUp , dDown = delta.copy(), delta.copy()
        dUp[dUp < 0] = 0
        dDown[dDown > 0] = 0
        RolUP = dUp.rolling(min_periods = n_days, window = n_days, center = False).mean()
        RolDown = dDown.abs().rolling(min_periods = n_days, window = n_days, center = False ).mean()

        subset['CMO']= (RolUP-RolDown)/(RolUP+RolDown)  * 100
        results.append(subset)

    return pd.concat(results)

df_CMO = CMO(df_RSI,code,n_days=14)

## BOP
def BOP(df):
    BOP= (df['종목종가']-df['종목시가'])/(df['종목고가'] - df['종목저가'])
    df['BOP'] = BOP
    return df

df_BOP = BOP(df_CMO)

## WILLR
def calculate_WILLR(group):
    low = group['종목저가'].rolling(window=14, min_periods=1).min()
    high = group['종목고가'].rolling(window=14, min_periods=1).max()
    close = group['종목종가']
    WILLR = ((high - close) / (high - low)) * (-100)
    group['WILLR'] = WILLR
    return group

WILLR = target.groupby('티커종목코드').apply(calculate_WILLR)

## STOCH_fastk
def calculate_stoch_fastk(group):
    low = group['종목저가'].rolling(window=14, min_periods=1).min()
    high = group['종목고가'].rolling(window=14, min_periods=1).max()
    close = group['종목종가']
    stoch_fastk = ((close - low) / (high - low)) * 100
    group['STOCH_fastk'] = stoch_fastk
    return group

STOCH_fastk = target.groupby('티커종목코드').apply(calculate_stoch_fastk)

In [12]:
df_CMO.reset_index(inplace=True)
df_CMO.rename(columns={'index': '거래일자'}, inplace=True)
WILLR.reset_index(inplace=True)
WILLR.rename(columns={'index': '거래일자'}, inplace=True)
STOCH_fastk.reset_index(inplace=True)
STOCH_fastk.rename(columns={'index': '거래일자'}, inplace=True)

merged_df2 = pd.merge(df_CMO, WILLR, on=['거래일자', '티커종목코드'], how='left', suffixes=('', '_WILLR'))
merged_df3 = pd.merge(merged_df2, STOCH_fastk, on=['거래일자', '티커종목코드'], how='left', suffixes=('', '_STOCH_fastk'))

In [13]:
# RSI 매도/매수/중립 판단
merged_df3['RSI_result'] = np.where((0 <= merged_df3['RSI']) & (merged_df3['RSI'] <= 30), 1,
                            np.where((70 <= merged_df3['RSI']) & (merged_df3['RSI'] <= 100), 0, 2))

# CMO 매도/매수/중립 판단
merged_df3['CMO_result'] = np.where((merged_df3['CMO'] <= -40), 1,
                            np.where((40<=merged_df3['CMO']), 0, 2))

# BOP 매도/매수/중립 판단
merged_df3['BOP_result'] = np.where((merged_df3['BOP'] > 0), 1,
                            np.where((0> merged_df3['BOP']), 0,2 ))

# STOCH_fastk 매도/매수/중립 판단
subset = merged_df3[['티커종목코드', 'STOCH_fastk']].copy()
subset['yesterday_STOCH_fastk'] = subset.groupby('티커종목코드')['STOCH_fastk'].shift(1)
subset['buy_condition'] = (subset['yesterday_STOCH_fastk'] <= 20.0) & (subset['STOCH_fastk'] > subset['yesterday_STOCH_fastk'])
subset['sell_condition'] = (subset['yesterday_STOCH_fastk'] >= 80.0) & (subset['STOCH_fastk'] < subset['yesterday_STOCH_fastk'])

subset['stock_fastk_result'] = 2
subset.loc[subset['buy_condition'], 'stock_fastk_result'] = 1
subset.loc[subset['sell_condition'], 'stock_fastk_result'] = 0

merged_df3['stock_fastk_result'] = subset['stock_fastk_result']

# WILLR 매도/매수/중립 판단
subset = merged_df3[['티커종목코드', 'WILLR']].copy()
subset['yesterday_WILLR'] = subset.groupby('티커종목코드')['WILLR'].shift(1)
subset['buy_condition'] = (subset['yesterday_WILLR'] <= -80.0) & (subset['WILLR'] > subset['yesterday_WILLR'])
subset['sell_condition'] = (-20<=subset['yesterday_WILLR']) & (subset['yesterday_WILLR'] <=0) & (subset['WILLR'] < subset['yesterday_WILLR'])

subset['WILLR_result'] = 2
subset.loc[subset['buy_condition'], 'WILLR_result'] = 1
subset.loc[subset['sell_condition'], 'WILLR_result'] = 0

merged_df3['WILLR_result'] = subset['WILLR_result']

In [14]:
target = merged_df3
target = target[['거래일자', '티커종목코드', 'RSI_result', 'CMO_result', 'BOP_result', 'stock_fastk_result', 'WILLR_result']]
target['거래일자'] = pd.to_datetime(target['거래일자'], format='%Y-%m-%d')
target['거래일자'] = pd.to_datetime(target['거래일자'], format='%Y%m%d')

In [15]:
# 5개의 기술지표를 종합하여 과반수인 것을 채택, 중복이 있다면 중립으로 판단

target['result'] = ""

for i in range(len(target)):
    results = []
    dup = []

    results.append(target['RSI_result'][i])
    results.append(target['CMO_result'][i])
    results.append(target['BOP_result'][i])
    results.append(target['stock_fastk_result'][i])
    results.append(target['WILLR_result'][i])

    counts = Counter(results)
    max_count = max(counts.values())
    for element, count in counts.items():
        if count == max_count:
            dup.append(str(element))

    if len(dup) == 1:
        majority_element = max(counts, key=counts.get)
        target.iloc[i,7] = majority_element

    elif len(dup) > 1:
        target.iloc[i,7] = 2

In [16]:
target = target[~(target['거래일자'].dt.month == 12)]
target.drop_duplicates(inplace=True)
target = target.reset_index(drop=True)

In [17]:
# 뉴스 발행일 다음 날 관련 종목의 기술지표 정보와 해당 뉴스를 매칭

df_news['rgs_dt'] = pd.to_datetime(df_news['rgs_dt'], format='%Y%m%d')
df_news['target'] = ""

df_news['adjusted_date'] = df_news['rgs_dt'] + pd.DateOffset(days=1)
merged_df = pd.merge(df_news, target, how='left', left_on=['tck_iem_cd', 'adjusted_date'], right_on=['티커종목코드', '거래일자'])

df_news['target'] = merged_df['result'].fillna("").values

df_news.drop('adjusted_date', axis=1, inplace=True)
if '거래일자' in df_news.columns:
    df_news.drop('거래일자', axis=1, inplace=True)

df_news = df_news[df_news['target'] != '']
df_news['target'] = df_news['target'].astype(int)

### 모델 훈련 및 평가

In [18]:
# 중립인 것은 제외
df_news = df_news[df_news['target'] != 2]

# 텍스트 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_news['combined_news'])
total_words = len(tokenizer.word_index) + 1

# 텍스트를 정수 시퀀스로 변환하고 패딩
input_sequences = tokenizer.texts_to_sequences(df_news['combined_news'])
max_sequence_length = max(len(sequence) for sequence in input_sequences)
padded_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)

# 타켓 데이터
labels = np.array(df_news['target'])

# Embedding 차원 정의
embedding_dim = 50

# 모델 구성
model = Sequential()
model.add(Embedding(total_words, embedding_dim, input_length=max_sequence_length))
model.add(GRU(dropout=0.5, units=100))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# 훈련 데이터와 검증 데이터 분리(7:3)
train_input, val_input, train_target, val_target = train_test_split(padded_sequences, labels, test_size=0.3, random_state=42)

# 모델 훈련
history = model.fit(train_input, train_target, epochs=6)

# 모델 평가
acc = model.evaluate(val_input, val_target)
print("Test Accuracy:", acc[1])
print("Test Loss:",acc[0])